In [20]:
# Install latest Classiq: pip install -U classiq
from classiq import *
from classiq import Parameter
import numpy as np
import matplotlib.pyplot as plt

# --- Parameters ---
NUM_QUBITS = 4
LAYERS = 5
MAX_ITERATIONS = 30

# --- Heisenberg Hamiltonian for 4-qubit chain ---
def create_heisenberg_hamiltonian():
    """Create Heisenberg Hamiltonian: H = J * sum(Xi*Xi+1 + Yi*Yi+1 + Zi*Zi+1)"""
    J = 1.0
    hamiltonian = []
    
    for i in range(NUM_QUBITS - 1):
        # X-X coupling
        pauli_x = [Pauli.I] * NUM_QUBITS
        pauli_x[i] = Pauli.X
        pauli_x[i + 1] = Pauli.X
        hamiltonian.append(PauliTerm(pauli_x, J))
        
        # Y-Y coupling
        pauli_y = [Pauli.I] * NUM_QUBITS
        pauli_y[i] = Pauli.Y
        pauli_y[i + 1] = Pauli.Y
        hamiltonian.append(PauliTerm(pauli_y, J))
        
        # Z-Z coupling
        pauli_z = [Pauli.I] * NUM_QUBITS
        pauli_z[i] = Pauli.Z
        pauli_z[i + 1] = Pauli.Z
        hamiltonian.append(PauliTerm(pauli_z, J))
    
    return hamiltonian

# --- Sz-Conserving Ansatz ---
@qfunc
def sz_conserving_layer(q: QArray[QBit], params: CArray[Parameter, NUM_QUBITS]) -> None:
    """Single layer of Sz-conserving ansatz"""
    # Apply RY rotations (preserve Sz)
    repeat(q.len, lambda i: RY(params[i], q[i]))
    
    # Entangling gates that preserve Sz (neighboring pairs)
    repeat(q.len // 2, lambda i: CX(q[2*i], q[2*i + 1]))
    repeat((q.len - 1) // 2, lambda i: CX(q[2*i + 1], q[2*i + 2]))

# Required main entry point for Classiq platform
@qfunc
def main(q: Output[QArray[QBit]]) -> None:
    """Main entry point required by Classiq platform"""
    allocate(NUM_QUBITS, q)
    
    # Initialize to |0,1,0,1⟩ for Sz = 0
    X(q[1])
    X(q[3])
    
    # Apply layers with parameterized gates
    for layer in range(LAYERS):
        # Create parameter names instead of CReal objects
        params = [Parameter(name=f"theta_{layer}_{i}") for i in range(NUM_QUBITS)]
        sz_conserving_layer(q, params)

# --- Hardware-Efficient Ansatz ---
@qfunc 
def hw_efficient_ansatz(q: Output[QArray[QBit]]) -> None:
    """Hardware-efficient ansatz with full connectivity"""
    allocate(NUM_QUBITS, q)
    
    # Apply layers of RY + entangling
    for layer in range(LAYERS):
        # Single-qubit rotations
        repeat(q.len, lambda i: RY(Parameter(name=f"ry_{layer}_{i}"), q[i]))
        
        # Entangling gates (full connectivity)
        for i in range(NUM_QUBITS):
            for j in range(i + 1, NUM_QUBITS):
                CX(q[i], q[j])

# --- VQE Execution ---
def run_vqe_comparison():
    """Run VQE for both ansatzes and compare results"""
    hamiltonian = create_heisenberg_hamiltonian()
    
    # Results storage
    results = {}
    
    # --- Sz-Conserving VQE ---
    print("Running Sz-conserving VQE...")
    sz_qmod = create_model(main)
    sz_qprog = synthesize(sz_qmod)
    
    # VQE for Sz-conserving
    vqe_result_sz = VQE(
        hamiltonian=hamiltonian,
        maximize=False,
        initial_point=None,
        optimizer=Optimizer.COBYLA,
        max_iteration=MAX_ITERATIONS,
        tolerance=1e-6
    ).optimize(sz_qprog)
    
    results['sz_conserving'] = {
        'energy': vqe_result_sz.energy,
        'convergence': vqe_result_sz.convergence_graph,
        'iterations': vqe_result_sz.iteration_count
    }
    
    # --- Hardware-Efficient VQE ---
    print("Running hardware-efficient VQE...")
    hw_qmod = create_model(hw_efficient_ansatz)
    hw_qprog = synthesize(hw_qmod)
    
    # VQE for hardware-efficient
    vqe_result_hw = VQE(
        hamiltonian=hamiltonian,
        maximize=False,
        initial_point=None,
        optimizer=Optimizer.COBYLA,
        max_iteration=MAX_ITERATIONS,
        tolerance=1e-6
    ).optimize(hw_qprog)
    
    results['hw_efficient'] = {
        'energy': vqe_result_hw.energy,
        'convergence': vqe_result_hw.convergence_graph,
        'iterations': vqe_result_hw.iteration_count
    }
    
    return results

# --- Plotting ---
def plot_comparison(results):
    """Plot energy convergence comparison"""
    plt.figure(figsize=(10, 6))
    
    # Extract convergence data
    sz_energies = results['sz_conserving']['convergence']
    hw_energies = results['hw_efficient']['convergence']
    
    plt.plot(range(len(sz_energies)), sz_energies, 
             'b-', label='Sz-Conserving Ansatz', linewidth=2)
    plt.plot(range(len(hw_energies)), hw_energies, 
             'r-', label='Hardware-Efficient Ansatz', linewidth=2)
    
    plt.xlabel('Iteration')
    plt.ylabel('Energy Expectation Value')
    plt.title('VQE Convergence: Sz-Conserving vs Hardware-Efficient Ansatz')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print final results
    print(f"\nFinal Results:")
    print(f"Sz-Conserving: {results['sz_conserving']['energy']:.6f} ({results['sz_conserving']['iterations']} iterations)")
    print(f"Hardware-Efficient: {results['hw_efficient']['energy']:.6f} ({results['hw_efficient']['iterations']} iterations)")

# --- Main Execution ---
if __name__ == "__main__":

    
    try:
        results = run_vqe_comparison()
        plot_comparison(results)
        print("VQE comparison completed successfully!")
        
    except Exception as e:
        print(f"Error during execution: {e}")
        print("Make sure you have authenticated with Classiq using authenticate()")

Running Sz-conserving VQE...
Error during execution: 
	ClassiqValueError: CArray accepts one or two generic parameters in the form `CArray[<element-type>]` or `CArray[<element-type>, <size>]` in function main
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack or open a support ticket at: https://classiq-community.freshdesk.com/support/tickets/new
Make sure you have authenticated with Classiq using authenticate()


In [10]:
authenticate()

/Users/shivasai/Desktop/classiq-library/.venv/lib/python3.12/site-packages/classiq/_internals/authentication/token_manager.py:101: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(
